# redundant and hacky implementation of BFS just to get some sample data
- idea is to start at URL, then explore every link on page until depth is reached. 
- in dire need of refactoring, right now it has some major redundancies
- note that links are only visited once to avoide cycles
- this is an edit

In [1]:
from selenium import webdriver
import pandas as pd


def validate_url(driver, url):
    ''' if valid url, return set (ie unique) of links otherwize return None'''
    try:
        driver.get(url)
        ids =  driver.find_elements_by_xpath('//*[@href]')
        return set([x.get_attribute('href') for x in ids])
    except:
        return set()

    
def bfs(seed_url, driver, frames, limit):
    ''' get all links from a given url'''
    # set params for DF organization
    depth = 0
    node_id = 0
    parent = 0
    # dont repeat (endless cycles)
    visited_set = set([seed_url])
    #initilize seed row
    frames[depth] = pd.DataFrame(dict(depth=[depth], node_id=[node_id], parent=[parent], url=[seed_url]))
    depth += 1
    # get links for seed
    links = validate_url(driver, seed_url)
    # remove ones that have been seen
    links = links - visited_set
    # used for unique identier of node for graphing
    new_node_id = len(links) + node_id + 1
    # build dataframe and append to list of frames
    temp_frame = pd.DataFrame(dict(node_id=list(range(node_id + 1,new_node_id)), url=list(links)))
    temp_frame['depth'] = depth
    temp_frame['parent'] = parent
    frames[depth] = (temp_frame)
    #update node id
    node_id = new_node_id - 1
    # update visited set
    visited_set.update(links)

    for i in range(depth, limit):
        depth += 1
        temp_level = list()
        for _,row in frames[i].iterrows():
            links = validate_url(driver, row['url'])
            links = links - visited_set
            if len(links) > 0:
                new_node_id = len(links) + node_id + 1
                temp_frame = pd.DataFrame(dict(node_id=list(range(node_id + 1, new_node_id)), url=list(links)))
                temp_frame['depth'] = depth
                temp_frame['parent'] = row['node_id']
                temp_level.append(temp_frame)
                node_id = new_node_id - 1
                visited_set.update(links)
            
        frames[depth] = pd.concat(temp_level)
            
    return pd.concat(frames.values())

# look at data from my website (not many links and still graph grows huge! )
- just using my personal site bc I know it doesnt have a ton of links
- could consider filtering link types (ie not getting resources files), this could be a simple as a regex for http

In [2]:
driver = webdriver.Firefox()
driver.set_page_load_timeout(10)
frames = dict()
tr = bfs("https://www.cas-donoghue.org/", driver, frames, 2)

driver.close()
tr.to_csv("look.csv")
tr.head()

,depth,node_id,parent,url
0,0,0,0,https://www.cas-donoghue.org/
0,1,1,0,https://donoghuc.github.io/
1,1,2,0,https://www.cas-donoghue.org/static/css/landin...
2,1,3,0,https://github.com/donoghuc
3,1,4,0,https://stackoverflow.com/users/9034974/cas-do...


# build node locations for visualization
- can optimize here for sure, idea is to have everythin centered on x axis at 0
- add columns that represent x and y coord of the nodes (urls)

In [3]:
def make_x_list(val):
    '''spread x coord around 0'''
    x = int(val/2)
    if val%2 != 0:
        return list(range(x*-1, x + 1))
    else:
        return list(range(x*-1, x))

tr['node_y'] = tr['depth'] * -1
count_series = tr['depth'].value_counts()
index_dict = dict()
y_col = list()

for i,v in count_series.iteritems():
    index_dict[i] = make_x_list(v)

for key in sorted(index_dict.keys()):
    y_col.extend(index_dict[key])
tr['node_x'] = y_col
  
tr.head()

,depth,node_id,parent,url,node_y,node_x
0,0,0,0,https://www.cas-donoghue.org/,0,0
0,1,1,0,https://donoghuc.github.io/,-1,-9
1,1,2,0,https://www.cas-donoghue.org/static/css/landin...,-1,-8
2,1,3,0,https://github.com/donoghuc,-1,-7
3,1,4,0,https://stackoverflow.com/users/9034974/cas-do...,-1,-6


# Build edge locations for visualizations
- want the edges to be color coded by parent, implement as a multi-line glyph and turn off hover

In [4]:
from bokeh.palettes import inferno
# get coordinants for parent nodes
parent_coords = dict()
for i in tr['parent'].unique():
    x = tr[tr['node_id'] == i]['node_x'].values[0]
    y = tr[tr['node_id'] == i]['node_y'].values[0]
    parent_coords[i] = (x,y)
# color code lines
colors = iter(inferno(len(parent_coords)))
color_dict = dict()
for key in parent_coords.keys():
    color_dict[key] = next(colors)
# get lists for edges
all_x = list()
all_y = list()
edge_colors = list()
for _,row in tr.iterrows():
    pcoord_tuple = parent_coords[row['parent']]
    all_x.append([row['node_x'], pcoord_tuple[0]])
    all_y.append([row['node_y'], pcoord_tuple[1]])
    edge_colors.append(color_dict[row['parent']])


# make one giant HTML file LOLOLOL
- will have to refactor to make this way smaller, for now cram all resources (JS, CSS) into one giant HTML
- for now getting a warning, will investigate later. 

In [5]:
from bokeh.plotting import figure, show, ColumnDataSource, output_file
from bokeh.models import HoverTool

# from bokeh.io import output_notebook
# output_notebook()

output_file('dfs_mock.html')
source = ColumnDataSource(data=dict(node_x=tr['node_x'], node_y=tr['node_y'], desc=tr['url']))

hover = HoverTool(tooltips=[('URL', "@desc")], names=['nodes'])

p = figure(plot_width=1200, plot_height=900, title="BFS prototype")
p.circle('node_x', 'node_y', name='nodes', size=5, source=source)
p.multi_line(all_x, all_y, color=edge_colors, line_width=1)
p.add_tools(hover)
show(p)

/home/cas/working_dir/visual_web_crawler/crawler/lib/python3.6/site-packages/bokeh/core/json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):
